<h1 style="text-align: center; font-family: Verdana; font-size: 32px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; font-variant: small-caps; letter-spacing: 3px; color: #74d5dd; background-color: #ffffff;">Human Protein Atlas - Single Cell Classification</h1>
<h2 style="text-align: center; font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: underline; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">Exploratory Data Analysis (EDA)</h2>
<h5 style="text-align: center; font-family: Verdana; font-size: 12px; font-style: normal; font-weight: bold; text-decoration: None; text-transform: none; letter-spacing: 1px; color: black; background-color: #ffffff;">CREATED BY: DARIEN SCHETTLER</h5>


<h2 style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;">TABLE OF CONTENTS</h2>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#imports">0&nbsp;&nbsp;&nbsp;&nbsp;IMPORTS</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#background_information">1&nbsp;&nbsp;&nbsp;&nbsp;BACKGROUND INFORMATION</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#setup">2&nbsp;&nbsp;&nbsp;&nbsp;SETUP</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#helper_functions">3&nbsp;&nbsp;&nbsp;&nbsp;HELPER FUNCTIONS</a></h3>

---

<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#helper_functions">4&nbsp;&nbsp;&nbsp;&nbsp;DATASET PREPERATION</a></h3>

---


<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#helper_functions">5&nbsp;&nbsp;&nbsp;&nbsp;MODEL BUILDING</a></h3>

---


<h3 style="text-indent: 10vw; font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;"><a href="#helper_functions">6&nbsp;&nbsp;&nbsp;&nbsp;MODEL TRAINING</a></h3>

---

<a style="text-align: font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; background-color: #ffffff; color: navy;" id="imports">0&nbsp;&nbsp;IMPORTS</a>

In [ ]:
print("\n... IMPORTS STARTING ...\n")
print("\n\tVERSION INFORMATION")

# For later visualization
!pip install -q visualkeras
import visualkeras

# Machine Learning and Data Science Imports
import tensorflow_addons as tfa; print(f"\t\t– TENSORFLOW ADDONS VERSION: {tfa.__version__}");
import tensorflow as tf; print(f"\t\t– TENSORFLOW VERSION: {tf.__version__}");
import pandas as pd; pd.options.mode.chained_assignment = None;
import numpy as np; print(f"\t\t– NUMPY VERSION: {np.__version__}");
import scipy; print(f"\t\t– SCIPY VERSION: {scipy.__version__}");

# Built In Imports
from kaggle_datasets import KaggleDatasets
from collections import Counter
from datetime import datetime
import multiprocessing
from glob import glob
import warnings
import requests
import imageio
import IPython
import urllib
import zipfile
import pickle
import random
import shutil
import string
import math
import tqdm
import time
import gzip
import io
import os
import gc
import re

# Visualization Imports
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import plotly.express as px
import seaborn as sns
from PIL import Image
import matplotlib; print(f"\t\t– MATPLOTLIB VERSION: {matplotlib.__version__}");
import plotly
import PIL
import cv2

# PRESETS
LBL_NAMES = ["Nucleoplasm", "Nuclear Membrane", "Nucleoli", "Nucleoli Fibrillar Center", "Nuclear Speckles", "Nuclear Bodies", "Endoplasmic Reticulum", "Golgi Apparatus", "Intermediate Filaments", "Actin Filaments", "Microtubules", "Mitotic Spindle", "Centrosome", "Plasma Membrane", "Mitochondria", "Aggresome", "Cytosol", "Vesicles", "Negative"]
INT_2_STR = {x:LBL_NAMES[x] for x in np.arange(19)}
INT_2_STR_LOWER = {k:v.lower().replace(" ", "_") for k,v in INT_2_STR.items()}
STR_2_INT_LOWER = {v:k for k,v in INT_2_STR_LOWER.items()}
STR_2_INT = {v:k for k,v in INT_2_STR.items()}
FIG_FONT = dict(family="Helvetica, Arial", size=14, color="#7f7f7f")
LABEL_COLORS = [px.colors.label_rgb(px.colors.convert_to_RGB_255(x)) for x in sns.color_palette("Spectral", len(LBL_NAMES))]
LABEL_COL_MAP = {str(i):x for i,x in enumerate(LABEL_COLORS)}

print("\n\n... IMPORTS COMPLETE ...\n")

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print("Running on TPU:", tpu.master())
except ValueError: # no TPU found, detect GPUs
    strategy = tf.distribute.get_strategy() # for GPU or multi-GPU machines
    print("\n... USING GPU ...\n")
    
N_REPLICAS = strategy.num_replicas_in_sync
print(f"... Number Of Accelerators: {N_REPLICAS} ...\n")

<a style="text-align: font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;" id="background_information">1&nbsp;&nbsp;XAI BACKGROUND INFORMATION</a>

<h3 style="text-align: font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">1.1  WHAT IS EXPLAINABLE AI - GENERAL INFO</h3>

---

For the purposes of this notebook and my explanation, I will be logically seperating explainable AI into two seperate branches. 

[**See this excerpt/paper that explains the branches in more detail.**](https://arxiv.org/pdf/1907.07374.pdf)

<i>
    
> "The two major categories presented here, namely perceptive interpretability and interpretability by mathematical structures, appear to present different polarities within the notion of interpretability. 
> 
> As an example for the difficulty with perceptive interpretability, when a visual evidence is given erroneously, the underlying mathematical structure may not seem to provide useful clues on the mistakes. 
> 
> On the other hand, a mathematical analysis of patterns may provide
information in high dimensions. They can only be easily perceived once the pattern is brought into lower dimensions, abstracting some fine-grained information we could not yet prove is not discriminative with measurable certainty."

[**<sup><sub>Tjoa, E., & Guan, C. (2019). A survey on explainable artificial intelligence (XAI): Towards
medical XAI. arXiv preprint arXiv:1907.0737</sub></sup>**](https://arxiv.org/pdf/1907.07374.pdf)

</i>
    
<br><br>

<b style="text-decoration: underline; font-family: Verdana;">PERCEPTIVE</b>
    
In short this is interpretability that can be observed by humans. Often the explanations arising through this branch are obvious to humans or already known.


<br>

<b style="text-decoration: underline; font-family: Verdana;">MATHEMATICAL</b>
    
In short this is interpretability that can only be observed by first applying mathematical manipulations to the data. An example technique that most are familiar with is clustering **[`(t-SNE)`](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding)**


<h3 style="text-align: font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">1.2  THE GOAL</h3>

---

**Pull the veil back on black-box machine learning models and help users understand how/why a model makes the decisions that it does. This can inform on how to improve the model as well as being useful for identifying things like bias and overfitting**

<center>

![XAI APPROACHES](https://i.ibb.co/ZXdBQ4D/Screen-Shot-2020-07-07-at-10-24-16-AM.png)

</center>

<h3 style="text-align: font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">1.3  CURRENT XAI APPROACHES</h3>

---

| Algorithm                     	| Type         	| Description                                                                                                                                                                                                                                                                                                                                                                                                  	|
|:-------------------------------	|:--------------	|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------	|
| Integrated Gradients        	| Gradient     	| Approximates the integral of gradients along the path (straight line from baseline to input) sand multiplies with (input - baseline)                                                                                                                                                                                                                                                                         	|
| DeepLift                    	| Application  	| Explains differences in the non-linear activations' outputs in terms of the differences of the input from its corresponding reference.                                                                                                                                                                                                   	|
| DeepLiftSHAP                	| Gradient     	| An extension of DeepLift that approximates SHAP values.<br>For each input example it considers a distribution of baselines and computes the expected value of the attributions based on DeepLift algorithm across all input-baseline pairs.                                                                 	|
| GradientSHAP                	| Gradient     	| Approximates SHAP values based on the expected gradients.<br>It adds gaussian noise to each input example #samples times, selects a random point between each sample and randomly drawn baseline from baselines' distribution, computes the gradient for it and multiples it with (input - baseline).<br>Final SHAP values represent the expected values of gradients * (input - baseline) for each input example. 	|
| Input * Gradient              	| Gradient     	| Multiplies model inputs with the gradients of the model outputs w.r.t. those inputs.                                                                                                                                                                                                                                                                                                                         	|
| Saliency                     	| Gradient     	| The gradients of the output w.r.t. inputs.                                                                                                                                                                                                                                                                                                                                                                   	|
| Guided BackProp / DeconvNet 	| Gradient     	| Computes the gradients of the model outputs w.r.t. its inputs.<br>If there are any RELUs present in the model, their gradients will be overridden so that only positive gradients of the inputs (in case of Guided BackProp) and outputs (in case of deconvnet) are back-propagated.                                                                                                                            	|
| Guided GradCam                	| Gradient     	| Computes the element-wise product of Guided BackProp and up-sampled positive GradCam attributions.                                                                                                                                                                                                                                                                                                           	|
| LayerGradCam                  	| Gradient     	| Computes the gradients of model outputs w.r.t. selected input layer, averages them for each output channel and multiplies with the layer activations.                                                                                                                                                                                                                                                        	|
| Layer Internal Influence      	| Gradient     	| Approximates the integral of gradients along the path from baseline to inputs for selected input layer.                                                                                                                                                                                                                                                                                                      	|
| Layer Conductance            	| Gradient     	| Decomposes integrated gradients via chain rule.<br>It approximates the integral of gradients defined by a chain rule, described as the gradients of the output w.r.t. to the neurons multiplied by the gradients of the neurons w.r.t. the inputs, along the path from baseline to inputs.<br>Finally, the latter is multiplied by (input - baseline).                                                             	|
| Layer Gradient * Activation   	| Gradient     	| Computes element-wise product of layer activations and the gradient of the output w.r.t. that layer.                                                                                                                                                                                                                                                                                                         	|
| Layer Activation              	| -            	| Computes the inputs or outputs of selected layer.                                                                                                                                                                                                                                                                                                                                                            	|
| Feature Ablation            	| Perturbation 	| Assigns an importance score to each input feature based on the magnitude changes in model output or loss when those features are replaced by a baseline (usually zeros) based on an input feature mask.                                                                                                                                                                                                      	|
| Feature Permutation           	| Perturbation 	| Assigns an importance score to each input feature based on the magnitude changes in model output or loss when those features are permuted based on input feature mask.                                                                                                                                                                                                                                       	|
| Occlusion                     	| Perturbation 	| Assigns an importance score to each input feature based on the magnitude changes in model output when those features are replaced by a baseline (usually zeros) using rectangular sliding windows and sliding strides.<br>If a features is located in multiple hyper-rectangles the importance scores are averaged across those hyper-rectangles.                                                               	|
| Shapely Value                 	| Perturbation 	| Computes feature importances based on all permutations of all input features.<br>It adds each feature for each permutation one-by-one to the baseline and computes the magnitudes of output changes for each feature which are ultimately being averaged across all permutations to estimate final attribution score.                                                                                           	|
| Shapely Value Sampling        	| Perturbation 	| Similar to Shapely value, but instead of considering all feature permutations it considers only #samples random permutations.                                                                                                                                                                                                                                                                                	|
| NoiseTunnel                   	| -            	| Depends on the choice of above mentioned attribution algorithm                                                                                                                                                                                                                                                                                                                                                                        	|


<h3 style="text-align: font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: none; letter-spacing: 2px; color: navy; background-color: #ffffff;">1.4  WHERE IS XAI NEEDED?</h3>

---

***Obviously things like the weakly-supervised tasks in this competition may require XAI***

XAI can be used for a wide range of things that we won't get into here (protecting against bias, protecting against overfitting, detecting features, etc.)

One other place XAI can be used is when working with black-box models. To understand what that is we will see the definitions and examples of the terms: **Transparent and Black-Box Models**:

<br>

<b style="text-decoration: underline; font-family: Verdana;">TRANSPARENT MODELS</b>

These are models/algorithms that are easily interpretable and **DO NOT** (generally) requre XAI. 

*Although occasionally post-hoc analysis is required or basic explainability tools.*

<b>

* Linear/Logistic Regression
* Decision Trees
* K-Nearest Neighbors
* Rule Based Learners
* General Additive Models
* Bayesian Models

</b><br>

<b style="text-decoration: underline; font-family: Verdana;">BLACK-BOX MODELS</b>

These are models/algorithms that are NOT easily interpretable and **DO** requre XAI. 

*This is not an exhaustive list of black-box models. It is simply the more common black-box models.*


<b>

* Tree Ensembles
* Support Vector Machines
* Multi-Layer Neural Network (MLPNN)
* Convolutional Neural Network (CNN)
* Recurrent Neural Network (RNN)

</b>

<h3 style="text-align: font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: uppercase; letter-spacing: 2px; color: navy; background-color: #ffffff;">1.4  Why Is XAI Needed: The Case For Growing Global AI Regulation</h3>

---

Many regulatory bodies have begun to encourage or enforce explainability in predictive algorithms used in the public domain.<br><br>**See below!**<br><sub>*(list was created roughly a year ago)*</sub>

- GDPR: Article 22 empowers individuals with the right to demand an explanation of how an
automated system made a decision that affects them.
- Algorithmic Accountability Act 2019: Requires companies to provide an assessment of the risks posed by
the automated decision system to the privacy or security and the risks that contribute to inaccurate, unfair,
biased, or discriminatory decisions impacting consumers
- California Consumer Privacy Act: Requires companies to rethink their approach to capturing,
storing, and sharing personal data to align with the new requirements by January 1, 2020.
- Washington Bill 1655: Establishes guidelines for the use of automated decision systems to protect
consumers, improve transparency, and create more market predictability.
- Massachusetts Bill H.2701: Establishes a commission on automated decision-making,
transparency, fairness, and individual rights.
- Illinois House Bill 3415: States predictive data analytics determining creditworthiness or hiring
decisions may not include information that correlates with the applicant race or zip code.

<a style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;" id="setup">2&nbsp;&nbsp;INTEGRATED GRADIENTS BACKGROUND INFORMATION</a>

This notebook will show how to implement Integrated Gradients (IG) for this competition**. IG is an Explainable AI (XAI) technique introduced in the paper **[Axiomatic Attribution for Deep Networks](https://arxiv.org/abs/1703.01365)


***LINKS***

&nbsp;&nbsp;&nbsp;&nbsp;- **[Tensorflow - Google Colab This Is Heavily Based Off Of](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/interpretability/integrated_gradients.ipynb)**<br>

---

**I**ntegrated **G**radients (**IG**) aims to explain the relationship between a model's predictions in terms of its features. It has many use cases including understanding feature importances, identifying data skew, and debugging model performance.

**IG** has become a popular interpretability technique due to its broad applicability to any differentiable model (e.g. images, text, structured data), ease of implementation, theoretical justifications, and computational efficiency relative to alternative approaches that allows it to scale to large networks and feature spaces such as images.

Go to this notebook to see the implementation of IG. In it, we will walk through an implementation of **IG** step-by-step to understand the pixel feature importances of an image classifier. 

---

As an example, consider this **[image](https://commons.wikimedia.org/wiki/File:San_Francisco_fireboat_showing_off.jpg)** of a fireboat spraying jets of water. 

You would classify this image as a **fireboat** and might highlight the pixels making up the **boat** and **water cannons** as being important to your decision. 

Your model will also classify this image as a fireboat later on in this tutorial; however, does it highlight the same pixels as important when explaining its decision?

In the images below titled "**IG** Attribution Mask" and "Original + **IG** Mask Overlay" you can see that your model instead highlights (in purple) the pixels comprising the boat's **water cannons** and **jets of water** as being ***more important than the boat itself*** to its decision. 

How will your model generalize to new fireboats? What about fireboats without water jets? 

Read on to learn more about how **IG** works and how to apply **IG** to your models to better understand the relationship between their predictions and underlying features.

![IG Example](https://www.tensorflow.org/tutorials/interpretability/images/IG_fireboat.png)

<a style="font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;" id="setup">3&nbsp;&nbsp;NOTEBOOK SETUP</a>

In [ ]:
# Define the root data directory
GCS_DATA_DIR = KaggleDatasets().get_gcs_path("hpa-single-cell-image-classification")
LOCAL_DATA_DIR = "/kaggle/input/hpa-single-cell-image-classification"

# Define the paths to the training and testing tfrecord and image folders respectively
TRAIN_IMG_DIR = os.path.join(GCS_DATA_DIR, "train")
TRAIN_TFREC_DIR = os.path.join(GCS_DATA_DIR, "train_tfrecords")
TEST_IMG_DIR = os.path.join(GCS_DATA_DIR, "test")
TEST_TFREC_DIR = os.path.join(GCS_DATA_DIR, "test_tfrecords")

# Capture all the relevant full image paths
TRAIN_IMG_PATHS = tf.io.gfile.glob(os.path.join(TRAIN_IMG_DIR, '*.png'))
TEST_IMG_PATHS = tf.io.gfile.glob(os.path.join(TEST_IMG_DIR, '*.png'))
print(f"\n... Recall that 4 training images compose one example (R,G,B,Y) ...")
print(f"... \t– i.e. The first 4 training files are:")
for path in [x.rsplit('/',1)[1] for x in TRAIN_IMG_PATHS[:4]]: print(f"... \t\t– {path}")
print(f"\n... The number of training images is {len(TRAIN_IMG_PATHS)} i.e. {len(TRAIN_IMG_PATHS)//4} 4-channel images ...")
print(f"... The number of testing images is {len(TEST_IMG_PATHS)} i.e. {len(TEST_IMG_PATHS)//4} 4-channel images ...")

# Capture all the relevant full tfrec paths
TRAIN_TFREC_PATHS = tf.io.gfile.glob(os.path.join(TRAIN_TFREC_DIR, '*.tfrec'))
TEST_TFREC_PATHS = tf.io.gfile.glob(os.path.join(TEST_TFREC_DIR, '*.tfrec'))
print(f"\n... The number of training tfrecord files is {len(TRAIN_TFREC_PATHS)} ...")
print(f"... The number of testing tfrecord files is {len(TEST_TFREC_PATHS)} ...\n")

# Define paths to the relevant csv files
TRAIN_CSV = os.path.join(LOCAL_DATA_DIR, "train.csv")
SS_CSV = os.path.join(LOCAL_DATA_DIR, "sample_submission.csv")

# Create the relevant dataframe objects
train_df = pd.read_csv(TRAIN_CSV)
ss_df = pd.read_csv(SS_CSV)

print("\n\nTRAIN DATAFRAME\n\n")
display(train_df.head(3))

print("\n\nSAMPLE SUBMISSION DATAFRAME\n\n")
display(ss_df.head(3))

<a style="text-align: font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;" id="helper_functions">4&nbsp;&nbsp;HELPER FUNCTIONS</a>

In [ ]:
def load_image(img_id, img_dir):
    """ Load An Image Using ID and Directory Path - Composes 4 Individual Images """
    rgby = [
        np.asarray(Image.open(os.path.join(img_dir, img_id+f"_{c}.png")), np.uint8) \
        for c in ["red", "green", "blue", "yellow"]
    ]
    return np.stack(rgby, axis=-1)


def plot_rgb(arr, figsize=(12,12)):
    """ Plot 3 Channel Microscopy Image """
    plt.figure(figsize=figsize)
    plt.title(f"RGB Composite Image", fontweight="bold")
    plt.imshow(arr)
    plt.axis(False)
    plt.show()
    
    
def convert_rgby_to_rgb(arr):
    """ Convert a 4 channel (RGBY) image to a 3 channel RGB image.
    
    Advice From Competition Host/User: lnhtrang

    For annotation (by experts) and for the model, I guess we agree that individual 
    channels with full range px values are better. 
    In annotation, we toggled the channels. 
    For visualization purpose only, you can try blending the channels. 
    For example, 
        - red = red + yellow
        - green = green + yellow/2
        - blue=blue.
        
    Args:
        arr (numpy array): The RGBY, 4 channel numpy array for a given image
    
    Returns:
        RGB Image
    """
    
    rgb_arr = np.zeros_like(arr[..., :-1])
    rgb_arr[..., 0] = arr[..., 0]
    rgb_arr[..., 1] = arr[..., 1]+arr[..., 3]/2
    rgb_arr[..., 2] = arr[..., 2]
    
    return rgb_arr

def tif_gzip_to_png(tif_path):
    """Function to convert .tif.gz to .png and put it in the same folder
    
    Eg. for working in local work station
    
    Args:
        tif_path (str): Path to the tif zip file to be converted to png
        
    Returns:
        None; Zip will be unzipped to same directory as .tif zip exists    
    """
    
    png_path = pathlib.Path(tif_path.replace('.tif.gz','.png'))
    tf = gzip.open(tif_path).read()
    img = imageio.imread(tf, 'tiff')
    imageio.imwrite(png_path, img)
    
    
def download_and_convert_tifgzip_to_png(url, target_path):    
    """Function to convert .tif.gz to .png and put it in the same folder
    
    Args:
        url (str): Path to the url containing the tifgzip file
        target_path (str): Path to directory to unzip to
        
    Returns:
        None; Images are downloaded and unzipped    
    """
    
    r = requests.get(url)
    f = io.BytesIO(r.content)
    tf = gzip.open(f).read()
    img = imageio.imread(tf, 'tiff')
    imageio.imwrite(target_path, img)

    
def get_new_data():
    public_hpa_df = pd.read_csv('../input/publichpa/kaggle_2021.tsv',sep='\t',header=None)
    public_hpa_df.columns = ['Image', 'Label']
    colors = ['blue', 'red', 'green', 'yellow']
    save_dir = os.path.join(os.getcwd(),'publichpa')
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    for i, row in public_hpa_df.iterrows():
        try:
            img = row.Image
            for color in colors:
                img_url = f'{img}_{color}.tif.gz'
                save_path = os.path.join(save_dir,  f'{os.path.basename(img)}_{color}.png')
                download_and_convert_tifgzip_to_png(img_url, save_path)
                print(f'Downloaded {img_url} as {save_path}')    
        except:
            print(f'failed to download: {img}')
    return save_dir


def plot_ex(arr, figsize=(20,6), title=None, plot_merged=True, rgb_only=False):
    """ Plot 4 Channels Side by Side """
    if plot_merged and not rgb_only:
        n_images=5 
    elif plot_merged and rgb_only:
        n_images=4
    elif not plot_merged and rgb_only:
        n_images=4
    else:
        n_images=3
    plt.figure(figsize=figsize)
    if type(title) == str:
        plt.suptitle(title, fontsize=20, fontweight="bold")

    for i, c in enumerate(["Red Channel – Microtubles", "Green Channel – Protein of Interest", "Blue - Nucleus", "Yellow – Endoplasmic Reticulum"]):
        if not rgb_only:
            ch_arr = np.zeros_like(arr[..., :-1])        
        else:
            ch_arr = np.zeros_like(arr)
        if c in ["Red Channel – Microtubles", "Green Channel – Protein of Interest", "Blue - Nucleus"]:
            ch_arr[..., i] = arr[..., i]
        else:
            if rgb_only:
                continue
            ch_arr[..., 0] = arr[..., i]
            ch_arr[..., 1] = arr[..., i]
        plt.subplot(1,n_images,i+1)
        plt.title(f"{c.title()}", fontweight="bold")
        plt.imshow(ch_arr)
        plt.axis(False)
        
    if plot_merged:
        plt.subplot(1,n_images,n_images)
        
        if rgb_only:
            plt.title(f"Merged RGB", fontweight="bold")
            plt.imshow(arr)
        else:
            plt.title(f"Merged RGBY into RGB", fontweight="bold")
            plt.imshow(convert_rgby_to_rgb(arr))
        plt.axis(False)
        
    plt.tight_layout(rect=[0, 0.2, 1, 0.97])
    plt.show()
    
    
def flatten_list_of_lists(l_o_l):
    return [item for sublist in l_o_l for item in sublist]

def get_class_wts(df, low_idx=4):
    label_counts = Counter([c for sublist in df.Label.str.split("|").to_list() for c in sublist])
    low_val = sorted(label_counts.values())[low_idx-1] # Not the lowest as it is very underrepresented
    class_wts ={int(k):min(1.0, low_val/v) for k,v in label_counts.items()}
    return {i:class_wts[i] for i in sorted(class_wts)}

def decode_image(image_data, resize_to=(512,512)):
    image = tf.image.decode_png(image_data, channels=1)
    # explicit size needed for TPU
    image = tf.image.resize(image, resize_to) 
    return tf.cast(image, tf.float32)

def str_2_multi_hot_encoding(tfstring, n_classes=19):
    ragged_indices = tf.strings.to_number(tf.strings.split(tfstring, sep="|"), out_type=tf.int32)
    one_hot_stack = tf.one_hot(ragged_indices, depth=n_classes)
    return tf.reduce_max(one_hot_stack, axis=-2)


def decode(serialized_example):
    """ Parses a set of features and label from the given `serialized_example`.
        
        It is used as a map function for `dataset.map`

    Args:
        serialized_example (tf.Example): A serialized example containing the
            following features:
                – sensor_feature_0 – [int64]
                – sensor_feature_1 – [int64]
                – sensor_feature_2 – [int64]
        is_test (bool, optional): Whether to allow for the label feature
        
    Returns:
        A decoded tf.data.Dataset object representing the tfrecord dataset
    """
    # Defaults are not specified since both keys are required.
    feature_dict = {
        'image': tf.io.FixedLenFeature(shape=(), dtype=tf.string),
        'image_name': tf.io.FixedLenFeature(shape=(), dtype=tf.string),
        'target': tf.io.FixedLenFeature(shape=(), dtype=tf.string),
    }
    
  
    # Define a parser
    features = tf.io.parse_single_example(serialized_example, features=feature_dict)
        
    image = decode_image(features['image'])
    image_name = features["image_name"]
    label = str_2_multi_hot_encoding(features["target"])
    return image, image_name, label


<a style="text-align: font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;" id="dataset_preperation">5&nbsp;&nbsp;DATASET PREPERATION</a>

In [ ]:
N_EPOCHS=5
N_EX = len(train_df)
INPUT_SHAPE = (512,512,3)
BATCH_SIZE=16*strategy.num_replicas_in_sync
SHUFF_BUFF=200
LOAD_MODEL_PATH = "/kaggle/input/hpa-xai-ig-tfrecords-tpu-training/model" # None or 'path/to/model'

In [ ]:
# num_parallel_reads=None forces the order to be preserved
raw_train_ds = tf.data.TFRecordDataset(TRAIN_TFREC_PATHS, num_parallel_reads=None)

# See an example
for raw in raw_train_ds.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw.numpy())
    for k,v in example.features.feature.items():
        print(k)
        if k!="image":
            print(f"\t--> {v.bytes_list.value[0]}")
        else:
            print(f"\t-->{str(v.bytes_list.value[0][:25])+' ... '}")

In [ ]:
train_ds = raw_train_ds.map(decode)

# See examples
print("\n ... NOTICE THE IMAGES ARE CLUMPED TOGETHER BY CHANNEL FOR A GIVEN ID ...\n")
for i, (img, image_name, lbl) in enumerate(train_ds.take(4)):
    print(f"IMAGE SHAPE : {img.shape}")
    print(f"IMG #{i//4} -- IMAGE NAME  : {image_name.numpy().decode()}")
    print(f"IMAGE LABEL : {lbl}\n")

In [ ]:
#####################################################
# ANNOYINGLY THIS DOES NOT WORK AS THE QUADRUPLE OF #
# CHANNEL IMAGES IS IN A DIFFERENT ORDER EVERY TIME #
#####################################################
# red_train_ds = train_ds.shard(4, index=3)
# green_train_ds = train_ds.shard(4, index=2)
# blue_train_ds = train_ds.shard(4, index=0)
# yellow_train_ds = train_ds.shard(4, index=1)
#####################################################

red_train_ds = train_ds.filter(lambda x,y,z: tf.strings.regex_full_match(y, ".*red.*"))
green_train_ds = train_ds.filter(lambda x,y,z: tf.strings.regex_full_match(y, ".*green.*"))
blue_train_ds = train_ds.filter(lambda x,y,z: tf.strings.regex_full_match(y, ".*blue.*"))
yellow_train_ds = train_ds.filter(lambda x,y,z: tf.strings.regex_full_match(y, ".*yellow.*"))

print("\n ... NOTICE THE IMAGES ARE NOW IN THEIR OWN DATASET BY COLOR ...\n")
for (img_r, image_name_r, lbl_r), (img_b, image_name_b, lbl_b) in zip(red_train_ds.take(4), blue_train_ds.take(4)):
    print(f"IMAGE SHAPE : R={img_r.shape} - B={img_b.shape}")
    print(f"IMAGE NAME  : R={image_name_r.numpy().decode()} - B={image_name_b.numpy().decode()}")
    print(f"IMAGE LABEL : R={lbl_r} - B={lbl_b}")
    print()

In [ ]:
def preprocess_tfrec_ds(red, green, blue, yellow, drop_yellow=True, return_id=False):
    (ri, rn, rl), (gi, gn, gl), (bi, bn, bl), (yi, yn, yl) = red, green, blue, yellow
    if drop_yellow:
        combo_img = tf.stack([ri[..., 0], gi[..., 0], bi[..., 0]], axis=-1)
    else:
        combo_img = tf.stack([ri[..., 0], gi[..., 0], bi[..., 0], yi[..., 0]], axis=-1)
    
    if return_id:
        img_id = tf.strings.substr(rn, pos=0, len=36) # 36 is length of id (always)
        return combo_img, img_id, rl
    else:
        return combo_img, rl
    
train_ds = tf.data.Dataset.zip((red_train_ds, green_train_ds, blue_train_ds, yellow_train_ds)).map(preprocess_tfrec_ds)

print("\n\t\t... TRAIN EXAMPLES ...\n")
for x,y in train_ds.take(3):
    plot_ex(x.numpy().astype(np.uint8), title=f"LABELS = {[INT_2_STR[lbl] for lbl in np.where(y.numpy()==1)[0]]}", rgb_only=True)

In [ ]:
def augment(img_batch, lbl_batch):
    # SEEDING & KERNEL INIT
    SEED = tf.random.uniform((2,), minval=0, maxval=100, dtype=tf.dtypes.int32)
    K = tf.random.uniform((1,), minval=0, maxval=4, dtype=tf.dtypes.int32)[0]

    img_batch = tf.image.stateless_random_flip_left_right(img_batch, SEED)
    img_batch = tf.image.stateless_random_flip_up_down(img_batch, SEED)
    img_batch = tf.image.rot90(img_batch, K)
    
    img_batch = tf.image.stateless_random_saturation(img_batch, 0.9, 1.1, SEED)
    img_batch = tf.image.stateless_random_brightness(img_batch, 0.075, SEED)
    img_batch = tf.image.stateless_random_contrast(img_batch, 0.9, 1.1, SEED)    

    return img_batch, lbl_batch

# ############################################################################ #
# #### NOT IMPLEMENTING CURRENTLY AS ITS VERY SLOW AND TFRECORDS CONTAIN #### #
# #### OVERLAPPING IMAGE_IDS MAKE IT IMPOSSIBLE TO SPLIT OFF ONE OR TWO  #### #
# ####                      I WILL FIX THAT LATER                        #### #
# ############################################################################ #
# val_ds = train_ds.take(N_VAL)
# train_ds = train_ds.skip(N_VAL)
# val_ds = val_ds.batch(BATCH_SIZE) \
#                .prefetch(tf.data.AUTOTUNE)
# ############################################################################ #

train_ds = train_ds.repeat() \
                   .shuffle(SHUFF_BUFF) \
                   .batch(BATCH_SIZE) \
                   .map(augment, num_parallel_calls=tf.data.AUTOTUNE) \
                   .prefetch(tf.data.AUTOTUNE)
train_ds

<a style="text-align: font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;" id="model_creation">6&nbsp;&nbsp;MODEL CREATION</a>

<h3 style="text-align: font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: uppercase; letter-spacing: 2px; color: navy; background-color: #ffffff;">6.1 Model Paramaters/Configuration</h3>

---

In [ ]:
DROPOUT = 0.25
LR = 0.00075
OUTPUT_ACTIVIATION = "sigmoid"
LOSS_FN = "binary_crossentropy"

class_wts = get_class_wts(train_df)
print("\n ... CLASS WEIGHTING ...\n")
for k,v in class_wts.items(): print(f"\tCLASS #{k:<2} --> {v:.4f}")

<h3 style="text-align: font-family: Verdana; font-size: 20px; font-style: normal; font-weight: normal; text-decoration: none; text-transform: uppercase; letter-spacing: 2px; color: navy; background-color: #ffffff;">6.2 Model Creation & Initialization</h3>

---

The original image size from the EfficientNet paper for EfficientNetB2 is 260x260x3. We are however not bound by this and can use a smaller/larger size if we want. The original image sizes used for every version of EfficientNet are:

- EfficientNetB0 - (224, 224, 3)
- EfficientNetB1 - (240, 240, 3)
- EfficientNetB2 - (260, 260, 3)
- EfficientNetB3 - (300, 300, 3)
- EfficientNetB4 - (380, 380, 3)
- EfficientNetB5 - (456, 456, 3)
- EfficientNetB6 - (528, 528, 3)
- EfficientNetB7 - (600, 600, 3)

<br>

**I'LL BE USING `EfficientNetB2` WITH (512x512x3) INPUT**

In [ ]:
def get_backbone(efficientnet_name="efficientnet_b0", input_shape=(512,512,3), include_top=False, weights="imagenet", pooling="avg"):
    if "b0" in efficientnet_name:
        eb = tf.keras.applications.EfficientNetB0(
            include_top=include_top, weights=weights, pooling=pooling, input_shape=input_shape
            )
    elif "b1" in efficientnet_name:
        eb = tf.keras.applications.EfficientNetB1(
            include_top=include_top, weights=weights, pooling=pooling, input_shape=input_shape
            )
    elif "b2" in efficientnet_name:
        eb = tf.keras.applications.EfficientNetB2(
            include_top=include_top, weights=weights, pooling=pooling, input_shape=input_shape
            )
    elif "b3" in efficientnet_name:
        eb = tf.keras.applications.EfficientNetB3(
            include_top=include_top, weights=weights, pooling=pooling, input_shape=input_shape
            )
    elif "b4" in efficientnet_name:
        eb = tf.keras.applications.EfficientNetB4(
            include_top=include_top, weights=weights, pooling=pooling, input_shape=input_shape
            )
    elif "b5" in efficientnet_name:
        eb = tf.keras.applications.EfficientNetB5(
            include_top=include_top, weights=weights, pooling=pooling, input_shape=input_shape
            )
    elif "b6" in efficientnet_name:
        eb = tf.keras.applications.EfficientNetB6(
            include_top=include_top, weights=weights, pooling=pooling, input_shape=input_shape
            )
    elif "b7" in efficientnet_name:
        eb = tf.keras.applications.EfficientNetB7(
            include_top=include_top, weights=weights, pooling=pooling, input_shape=input_shape
            )
    else:
        raise ValueError("Invalid EfficientNet Name!!!")
    return eb


def add_head_to_bb(bb, n_classes=19, dropout=0.3):
    x = tf.keras.layers.Dropout(dropout)(bb.output)
    output = tf.keras.layers.Dense(n_classes, activation="softmax")(x)
    return tf.keras.Model(inputs=bb.inputs, outputs=output)


if LOAD_MODEL_PATH is None:
    with strategy.scope():
        eb = get_backbone("b5")
        eb = add_head_to_bb(eb)
        eb.compile(optimizer=tf.keras.optimizers.Adam(lr=LR), 
                   loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=0.0), 
                   metrics=["acc", tf.keras.metrics.AUC(name="auc", multi_label=True)])
else:
    with strategy.scope():
        load_locally = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
        eb = tf.keras.models.load_model(LOAD_MODEL_PATH, options=load_locally) # loading in Tensorflow's "SavedModel" format

print("\n... VISUAL OF MODEL ...\n")
display(visualkeras.layered_view(eb))

print("\n... MODEL SUMMARY ...\n")
eb.summary()

<a style="text-align: font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;" id="model_training">7&nbsp;&nbsp;MODEL TRAINING</a>

In [ ]:
### POTENTIAL LOSS FN ###
# def macro_double_soft_f1(y, y_hat):
#     """Compute the macro soft F1-score as a cost (average 1 - soft-F1 across all labels).
#     Use probability values instead of binary predictions.
#     This version uses the computation of soft-F1 for both positive and negative class for each label.
    
#     Args:
#         y (int32 Tensor): targets array of shape (BATCH_SIZE, N_LABELS)
#         y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        
#     Returns:
#         cost (scalar Tensor): value of the cost function for the batch
#     """
#     y = tf.cast(y, tf.float32)
#     y_hat = tf.cast(y_hat, tf.float32)
#     tp = tf.reduce_sum(y_hat * y, axis=0)
#     fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
#     fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
#     tn = tf.reduce_sum((1 - y_hat) * (1 - y), axis=0)
#     soft_f1_class1 = 2*tp / (2*tp + fn + fp + 1e-16)
#     soft_f1_class0 = 2*tn / (2*tn + fn + fp + 1e-16)
#     cost_class1 = 1 - soft_f1_class1 # reduce 1 - soft-f1_class1 in order to increase soft-f1 on class 1
#     cost_class0 = 1 - soft_f1_class0 # reduce 1 - soft-f1_class0 in order to increase soft-f1 on class 0
#     cost = 0.5 * (cost_class1 + cost_class0) # take into account both class 1 and class 0
#     macro_cost = tf.reduce_mean(cost) # average on all labels
#     return macro_cost

### POTENTIAL LOSS FN ###
# def macro_soft_f1(y, y_hat):
#     """Compute the macro soft F1-score as a cost.
#     Average (1 - soft-F1) across all labels.
#     Use probability values instead of binary predictions.
    
#     Args:
#         y (int32 Tensor): targets array of shape (BATCH_SIZE, N_LABELS)
#         y_hat (float32 Tensor): probability matrix of shape (BATCH_SIZE, N_LABELS)
        
#     Returns:
#         cost (scalar Tensor): value of the cost function for the batch
#     """
    
#     y = tf.cast(y, tf.float32)
#     y_hat = tf.cast(y_hat, tf.float32)
#     tp = tf.reduce_sum(y_hat * y, axis=0)
#     fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
#     fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
#     soft_f1 = 2*tp / (2*tp + fn + fp + 1e-16)
#     cost = 1 - soft_f1 # reduce 1 - soft-f1 in order to increase soft-f1
#     macro_cost = tf.reduce_mean(cost) # average on all labels
    
#     return macro_cost

### POTENTIAL METRIC ###
# def macro_f1(y, y_hat, thresh=0.5):
#     """Compute the macro F1-score on a batch of observations (average F1 across labels)
    
#     Args:
#         y (int32 Tensor): labels array of shape (BATCH_SIZE, N_LABELS)
#         y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
#         thresh: probability value above which we predict positive
        
#     Returns:
#         macro_f1 (scalar Tensor): value of macro F1 for the batch
#     """
#     y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
#     tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
#     fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
#     fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
#     f1 = 2*tp / (2*tp + fn + fp + 1e-16)
#     macro_f1 = tf.reduce_mean(f1)
#     return macro_f1


# Using an LR ramp up because fine-tuning a pre-trained model.
# Starting with a high LR would break the pre-trained weights.
LR_START = 0.00001
LR_MAX = 0.000125*strategy.num_replicas_in_sync
LR_MIN = 0.00001
LR_RAMPUP_EPOCHS = 3
LR_SUSTAIN_EPOCHS = 0
LR_EXP_DECAY = 0.875

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr

# VIEW SCHEDULE
rng = [i for i in range(10 if N_EPOCHS<10 else N_EPOCHS)]
y = [lrfn(x) for x in rng]

plt.figure(figsize=(10,4))
plt.plot(rng, y)
plt.title("CUSTOM LR SCHEDULE", fontweight="bold")
plt.show()

print(f"Learning rate schedule: {y[0]:.3g} to {max(y):.3g} to {y[-1]:.3g}")

In [ ]:
history = eb.fit(
    train_ds, 
    callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=1),
        tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2, verbose=1, restore_best_weights=True),
        tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=1)
    ], 
    class_weight=class_wts, 
    epochs=N_EPOCHS,
    steps_per_epoch=N_EX//BATCH_SIZE,
)
save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
eb.save('./model', options=save_locally) # saving in Tensorflow's "SavedModel" format

In [ ]:
plt.figure(figsize = (12, 12))

plt.subplot(2,1,1)
plt.xlabel("Epochs", fontweight="bold")
plt.ylabel("Loss", fontweight="bold")
plt.plot(history.history["loss"], label = "Training Loss" , marker='o')
plt.grid(True)
plt.legend()

plt.subplot(2,1,2)
plt.xlabel("Epochs")
plt.ylabel("AUC")
plt.plot(history.history["auc"], label = "Training AUC" , marker='o')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

<a style="text-align: font-family: Verdana; font-size: 24px; font-style: normal; font-weight: bold; text-decoration: none; text-transform: none; letter-spacing: 3px; color: navy; background-color: #ffffff;" id="appendix">8&nbsp;&nbsp;APPENDIX - HOW TO RECREATE THE TFRECORDS</a>

In [ ]:
def _bytes_feature(value, is_list=False):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    
    if not is_list:
        value = [value]
    
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))

def _float_feature(value, is_list=False):
    """Returns a float_list from a float / double."""
        
    if not is_list:
        value = [value]
        
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _int64_feature(value, is_list=False):
    """Returns an int64_list from a bool / enum / int / uint."""
        
    if not is_list:
        value = [value]
        
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def serialize(image, image_name, target):
    """
    Creates a tf.Example message ready to be written to a file from 4 features.

    Args:
        image (TBD): TBD
        image_name (str): TBD
        target (str): | delimited integers
    
    Returns:
        A tf.Example Message ready to be written to file
    """

    # Create a dictionary mapping the feature name to the 
    # tf.Example-compatible data type.
    feature = {
        'image': _bytes_feature(image, is_list=False),
        'image_name': _bytes_feature(image_name, is_list=False),
        'target': _bytes_feature(target, is_list=False),
    }

    # Create a Features message using tf.train.Example.
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()